# Bayesian optimization with parallel evaluation of an external objection function using Emukit

This tutorial will show you how to leverage Emukit to do Bayesian optimization on an external objective function that we can evaluate multiple times in parallel.

## Overview

By the end of the tutorial, you will be able to:

1. Generate batches $\{X_t | t \in 1..\}$ of objective function evaluation locations $\{x_i | x_i \in X_t\}$
2. Evaluate the objective function at these suggested locations in parallel $f(x_i)$
3. Use channels to implement the concurrency structure supporting this parallel evaluation

This tutorial requires basic familiarity with Bayesian optimization and concurrency. If you've never run Bayesian optimization using Emukit before, please refer to the [introductory tutorial](Emukit-tutorial-intro.ipynb) for more information. The concurrency used here is not particularly complicated, so you should be able to follow just fine without much more than an understanding of the [active object design pattern](https://en.wikipedia.org/wiki/Active_object).

The overview must start with the general imports and plots configuration

The overview section must finish with a Navigation that links to the main sections of the notebook


In [1]:
### General imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

### --- Figure config
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
LEGEND_SIZE = 15
TITLE_SIZE = 25
AXIS_SIZE = 15

### Navigation

1. [Section 1](#1.-Section-1)

2. [Section 2](#2.-Section-2)

3. [Conclusions](#3.-Conclusions)

4. [References](#4.-References)

## 1. Define objective function

In [11]:
# Specific imports that are used in a section should be loaded at the beginning of that section.
# It is ok if an import is repeated multiple times over the notebook

import time
import asyncio
import GPy
import emukit

from emukit.test_functions.branin import (
    branin_function as _branin_function,
)

In [21]:
### Define the cost and objective functions

_branin, _ps = _branin_function()

async def a_cost(x: np.ndarray):
    # Cost function, defined arbitrarily
    t = x.sum()/10
    await asyncio.sleep(t)

async def a_objective(x: np.ndarray):
    # Objective function
    r = _branin(x)
    await a_cost(x)
    return r

async def demo_async_obj():
    '''This function demonstrates a simple usage of the async objective function'''
    # Configure
    _x = [7.5, 12.5]
    d = len(_x)
    x = np.array(_x).reshape((1, d))
    assert _ps.check_points_in_domain(x).all(), ("You configured a point outside the objective"
        f"function's domain: {x} is outside {_ps.get_bounds()}")
    # Execute
    print(f"Input: x={x}")
    t0 = time.perf_counter()
    r = await a_objective(x)
    t1 = time.perf_counter()
    print(f"Output: result={r}")
    print(f"Time elapsed: {t1-t0} sec")

In [22]:
await demo_async_obj()

Input: x=[[ 7.5 12.5]]
Output: result=[[138.09715472]]
Time elapsed: 2.000624129000016 sec


Due to a [certain issue](https://github.com/spatialaudio/nbsphinx/issues/203) blockquotes cannot immediately follow code blocks, so make sure there is some text between a code block and a note. As soon as nbsphinx is updated with the [fix](https://github.com/spatialaudio/nbsphinx/pull/216) this can be relaxed.

>**Note**: Important notes in the text can be added like that
>
> notes can be multiline

Links to references done like that: [[Kennedy and O'Hagan, 2000]](#4.-References), the full reference should be included at the end of the notebook.

## 2. Run BO using parallel evaluation of batched suggestions

In [ ]:
### First, setup the BO



More section content

## 3. Conclusions

## 4. References

- Kennedy, M.C. and O'Hagan, A., 2000. *Predicting the output from a complex computer code when fast approximations are available.* Biometrika, 87(1), pp.1-13.